<a href=\"https://github.com/CERIA-Team/ceria-machinelearning/blob/main/capstone_test.ipynb" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>

In [ ]:
!pip install kaggle

import os
os.environ['KAGGLE_USERNAME'] = "pideeell"
os.environ['KAGGLE_KEY'] = "a917e24c299339667d5610765517f26b"

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
import pandas as pd
import joblib
import numpy as np

from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Music Dataset

In [ ]:
!kaggle datasets download -d maharshipandya/-spotify-tracks-dataset

Dataset URL: https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset
License(s): ODbL-1.0
 98% 8.00M/8.17M [00:01<00:00, 12.8MB/s]
100% 8.17M/8.17M [00:01<00:00, 7.89MB/s]


In [ ]:
!unzip '/content/-spotify-tracks-dataset.zip' -d '/content/dataset/'
!rm '/content/-spotify-tracks-dataset.zip'

Archive:  /content/-spotify-tracks-dataset.zip
  inflating: /content/dataset/dataset.csv  


In [ ]:
df = pd.read_csv('/content/dataset/dataset.csv')
df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

## Data Cleaning

In [ ]:
#select some columns from df
df = df[['track_id','track_name','track_genre','popularity','energy','tempo','valence']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   track_id     114000 non-null  object 
 1   track_name   113999 non-null  object 
 2   track_genre  114000 non-null  object 
 3   popularity   114000 non-null  int64  
 4   energy       114000 non-null  float64
 5   tempo        114000 non-null  float64
 6   valence      114000 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 6.1+ MB


In [ ]:
# print all unique genres
df['track_genre'].unique()

array(['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient',
       'anime', 'black-metal', 'bluegrass', 'blues', 'brazil',
       'breakbeat', 'british', 'cantopop', 'chicago-house', 'children',
       'chill', 'classical', 'club', 'comedy', 'country', 'dance',
       'dancehall', 'death-metal', 'deep-house', 'detroit-techno',
       'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm',
       'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk',
       'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove',
       'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle',
       'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian',
       'indie-pop', 'indie', 'industrial', 'iranian', 'j-dance', 'j-idol',
       'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino',
       'malay', 'mandopop', 'metal', 'metalcore', 'minimal-techno', 'mpb',
       'new-age', 'opera', 'pagode', 'party', 'piano', 'pop-film', 'pop',
       'pow

In [ ]:
#looking duplicates on track_id
df.duplicated(subset=['track_name']).sum()

40391

In [ ]:
#show duplicated value on track_name
df[df.duplicated(subset=['track_name'])]

,track_id,track_name,track_genre,popularity,energy,tempo,valence
18,2qLMf6TuEC3ruGJg4SMMN6,Lucky,acoustic,68,0.414,130.088,0.6690
20,3S0OXQeoh0w6AY8WQVckRW,I'm Yours,acoustic,75,0.444,150.960,0.7120
22,5TvE3pk05pyFIGdSY9j4DJ,Say Something,acoustic,70,0.147,141.284,0.0765
28,5QAMZTM5cmLg3fHX9ZbTZi,Winter Wonderland,acoustic,0,0.309,145.363,0.6640
29,2qESE1ZeWly7I3YjyTXmXh,Winter Wonderland,acoustic,0,0.309,145.363,0.6640
...,...,...,...,...,...,...,...
113974,0VVvjjtKyOeT2AEMTSQ33t,Faithful,world-music,34,0.419,131.973,0.3730
113980,64BdW5aHuB3C0QP3qjBJVi,Amen,world-music,41,0.634,76.994,0.0921
113987,4jDhzTYkEG5GloIWwVeVkc,All My Life - Live,world-music,40,0.938,128.002,0.4530
113991,0CE0Y6GM75cbrqao8EOAlW,At The Cross (Love Ran Red),world-music,32,0.531,146.003,0.1530


In [ ]:
#delete duplicates
df_clean = df.drop_duplicates(subset=['track_name'])
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73609 entries, 0 to 113999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   track_id     73609 non-null  object 
 1   track_name   73608 non-null  object 
 2   track_genre  73609 non-null  object 
 3   popularity   73609 non-null  int64  
 4   energy       73609 non-null  float64
 5   tempo        73609 non-null  float64
 6   valence      73609 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 4.5+ MB


In [ ]:
unique_genres = df_clean['track_genre'].unique()
for genre in unique_genres:
    print(genre)

acoustic
afrobeat
alt-rock
alternative
ambient
anime
black-metal
bluegrass
blues
brazil
breakbeat
british
cantopop
chicago-house
children
chill
classical
club
comedy
country
dance
dancehall
death-metal
deep-house
detroit-techno
disco
disney
drum-and-bass
dub
dubstep
edm
electro
electronic
emo
folk
forro
french
funk
garage
german
gospel
goth
grindcore
groove
grunge
guitar
happy
hard-rock
hardcore
hardstyle
heavy-metal
hip-hop
honky-tonk
house
idm
indian
indie-pop
indie
industrial
iranian
j-dance
j-idol
j-pop
j-rock
jazz
k-pop
kids
latin
latino
malay
mandopop
metal
metalcore
minimal-techno
mpb
new-age
opera
pagode
party
piano
pop-film
pop
power-pop
progressive-house
psych-rock
punk-rock
punk
r-n-b
reggae
reggaeton
rock-n-roll
rock
rockabilly
romance
sad
salsa
samba
sertanejo
show-tunes
singer-songwriter
ska
sleep
soul
spanish
study
swedish
synth-pop
tango
techno
trance
trip-hop
turkish
world-music


In [ ]:
import pandas as pd

# Create a dictionary to map genres to their respective heart rate intensities
genre_to_intensity = {
    'acoustic': {0, 1},
    'afrobeat': {1},
    'alt-rock': {2},
    'alternative': {1},
    'ambient': {0},
    'anime': {1, 2},
    'black-metal': {3},
    'bluegrass': {0},
    'blues': {0},
    'brazil': {1},
    'breakbeat': {3},
    'british': {1, 2}, # Broad genre, adjust as needed
    'cantopop': {1},
    'chicago-house': {2},
    'children': {0},
    'chill': {0},
    'classical': {0},
    'club': {2, 3}, # Broad, depends on specific subgenres
    'comedy': {0, 1},
    'country': {1},
    'dance': {1, 2},
    'dancehall': {2},
    'death-metal': {3},
    'deep-house': {2},
    'detroit-techno': {2, 3},
    'disco': {2},
    'disney': {1},
    'drum-and-bass': {2, 3},
    'dub': {1, 2},
    'dubstep': {2, 3},
    'edm': {2, 3},
    'electro': {2},
    'electronic': {2, 3},
    'emo': {1, 2},
    'folk': {0, 1},
    'forro': {2},
    'french': {1, 2}, # Broad genre, adjust as needed
    'funk': {1},
    'garage': {1, 2},
    'german': {1, 2}, # Broad genre, adjust as needed
    'gospel': {1},
    'goth': {3},
    'grindcore': {3},
    'groove': {1},
    'grunge': {2},
    'guitar': {0, 1, 2, 3}, # Extremely broad, adjust as needed
    'happy': {1},
    'hard-rock': {2, 3},
    'hardcore': {3},
    'hardstyle': {3},
    'heavy-metal': {3},
    'hip-hop': {1, 2, 3},
    'honky-tonk': {1},
    'house': {2, 3},
    'idm': {2},
    'indian': {1, 2}, # Broad genre, adjust as needed
    'indie-pop': {1},
    'indie': {1, 2},
    'industrial': {2, 3},
    'iranian': {1, 2}, # Broad genre, adjust as needed
    'j-dance': {2},
    'j-idol': {1, 2},
    'j-pop': {1},
    'j-rock': {2},
    'jazz': {0, 1},
    'k-pop': {1},
    'kids': {0},
    'latin': {1, 2},
    'latino': {1, 2},
    'malay': {1},
    'mandopop': {1},
    'metal': {2, 3},
    'metalcore': {3},
    'minimal-techno': {2},
    'mpb': {1},
    'new-age': {0},
    'opera': {0, 1}, # Can vary greatly in intensity
    'pagode': {2},
    'party': {2, 3},
    'piano': {0, 1},
    'pop-film': {1},
    'pop': {1, 2},
    'power-pop': {1},
    'progressive-house': {2},
    'psych-rock': {1, 2},
    'punk-rock': {2},
    'punk': {3},
    'r-n-b': {1, 2},
    'reggae': {1, 2},
    'reggaeton': {2},
    'rock-n-roll': {1, 2},
    'rock': {2, 3},
    'rockabilly': {1},
    'romance': {0, 1},
    'sad': {0, 1},
    'salsa': {2},
    'samba': {2},
    'sertanejo': {1, 2},
    'show-tunes': {1},
    'singer-songwriter': {0, 1},
    'ska': {2},
    'sleep': {0},
    'songwriter': {0},
    'soul': {1},
    'spanish': {1, 2}, # Broad genre, adjust as needed
    'study': {0},
    'swedish': {1, 2}, # Broad genre, adjust as needed
    'synth-pop': {1},
    'tango': {1, 2},
    'techno': {2, 3},
    'trance': {2, 3},
    'trip-hop': {1, 2},
    'turkish': {1, 2}, # Broad genre, adjust as needed
    'world-music': {0, 1, 2}
}

def assign_intensity(row):
    genre = row['track_genre'].lower()
    tempo = row['tempo']

    intensity_set = genre_to_intensity.get(genre)
    if intensity_set is None:
        print(f"Warning: Genre not found - '{row['track_name']}' - Genre: '{genre}'")
        return None

    # Find matching intensities based on tempo
    matched_intensities = set()
    if 0 in intensity_set and tempo <= 70:
        matched_intensities.add(0)
    if 1 in intensity_set and 80 <= tempo <= 120:
        matched_intensities.add(1)
    if 2 in intensity_set and 110 <= tempo <= 140:
        matched_intensities.add(2)
    if 3 in intensity_set and tempo > 130:
        matched_intensities.add(3)

    if matched_intensities:
        return matched_intensities
    else:
        print(f"Warning: Intensity not assigned - '{row['track_name']}' - Genre: '{genre}' - Tempo: {tempo}")
        return None

df_assign = df_clean

# Apply the function to each row to assign intensity
df_assign['intensity'] = df_clean.apply(assign_intensity, axis=1)

# # Drop rows where intensity could not be determined
# df_assign = df_assign.dropna(subset=['intensity'])

# # Ensure the intensity column is of integer type
# df_assign['intensity'] = df_assign['intensity'].astype(int)

# Display the first few rows of the dataframe to verify the result
print(df_assign.head(30))


Streaming output truncated to the last 5000 lines.
                  track_id                                   track_name  \
0   5SuOikwiRyPMVoIQDJUgSV                                       Comedy   
1   4qPNDBW1i3p13qLCt0Ki3A                             Ghost - Acoustic   
2   1iJBSr7s7jYXzM8EGcbK5b                               To Begin Again   
3   6lfxq3CG4xtTiEg7opyCyx                   Can't Help Falling In Love   
4   5vjLSffimiIP26QG5WcN2K                                      Hold On   
5   01MVOl9KtVTNfFiBU9I7dc                         Days I Will Remember   
6   6Vc5wAMmXdKIAM7WUoEb7N                                Say Something   
7   1EzrEOXmMH3G43AXT1y7pA                                    I'm Yours   
8   0IktbUcnAGrvD03AWnz3Q8                                        Lucky   
9   7k9GuJYLp2AzqokyEdwEw2                                       Hunger   
10  4mzP5mHkRvGxdhdGdAH7EJ                         Give Me Your Forever   
11  5ivF4eQBqJiVL5IAE9jRyl                       

<ipython-input-13-8de77565bb74>:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assign['intensity'] = df_clean.apply(assign_intensity, axis=1)


In [ ]:
# drop df_assign if intensity = None
df_assign = df_assign.dropna(subset=['intensity'])
df_assign.head(50)

,track_id,track_name,track_genre,popularity,energy,tempo,valence,intensity
0,5SuOikwiRyPMVoIQDJUgSV,Comedy,acoustic,73,0.4610,87.917,0.7150,{1}
4,5vjLSffimiIP26QG5WcN2K,Hold On,acoustic,82,0.4430,119.949,0.1670,{1}
5,01MVOl9KtVTNfFiBU9I7dc,Days I Will Remember,acoustic,58,0.4810,98.017,0.6660,{1}
10,4mzP5mHkRvGxdhdGdAH7EJ,Give Me Your Forever,acoustic,74,0.3630,99.905,0.3010,{1}
13,0X9MxHR1rTkEHDjp95F2OO,Bad Liar,acoustic,62,0.2340,87.103,0.2090,{1}
15,1KHdq8NK9QxnGjdXb55NiG,Falling in Love at a Coffee Shop,acoustic,58,0.5610,83.457,0.2380,{1}
16,6xKeQgzfjixSUld14qUezm,ily (i love you baby),acoustic,56,0.1120,110.154,0.4140,{1}
17,4Yo0igmcoNyat1secaH0OD,At My Worst,acoustic,54,0.0841,91.803,0.6090,{1}
19,6CgNoAbFJ4Q4Id4EjtbXlC,Photograph,acoustic,67,0.3200,107.946,0.3220,{1}
21,210JCw2LbYD4YIs8GiZ9iP,Demons,acoustic,63,0.3510,90.032,0.5690,{1}


In [ ]:
df_assign.head(-50)

,track_id,track_name,track_genre,popularity,energy,tempo,valence,intensity
0,5SuOikwiRyPMVoIQDJUgSV,Comedy,acoustic,73,0.461,87.917,0.715,{1}
4,5vjLSffimiIP26QG5WcN2K,Hold On,acoustic,82,0.443,119.949,0.167,{1}
5,01MVOl9KtVTNfFiBU9I7dc,Days I Will Remember,acoustic,58,0.481,98.017,0.666,{1}
10,4mzP5mHkRvGxdhdGdAH7EJ,Give Me Your Forever,acoustic,74,0.363,99.905,0.301,{1}
13,0X9MxHR1rTkEHDjp95F2OO,Bad Liar,acoustic,62,0.234,87.103,0.209,{1}
...,...,...,...,...,...,...,...,...
113893,3NjxQKRl1zjzHlOH0EVRBy,Pasión,world-music,23,0.865,131.984,0.444,{2}
113896,05W8PJmja6rf2jlXf57C5a,Secret Place - Live,world-music,37,0.297,129.829,0.146,{2}
113901,0k0Eiw6sJJUYyKcVzDILD4,Tethered,world-music,34,0.209,124.092,0.172,{2}
113902,7pYtl9OXETa8itI4iFjMhJ,Pursue,world-music,37,0.531,87.955,0.074,{1}


In [ ]:
df_assign.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36458 entries, 0 to 113997
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   track_id     36458 non-null  object 
 1   track_name   36458 non-null  object 
 2   track_genre  36458 non-null  object 
 3   popularity   36458 non-null  int64  
 4   energy       36458 non-null  float64
 5   tempo        36458 non-null  float64
 6   valence      36458 non-null  float64
 7   intensity    36458 non-null  object 
dtypes: float64(3), int64(1), object(4)
memory usage: 2.5+ MB


In [ ]:
# save df_assign as csv
df_assign.to_csv('/content/dataset/filtered_song.csv', index=False)

# Fitness Dataset (old)

In [ ]:
!kaggle datasets download -d aakashjoshi123/exercise-and-fitness-metrics-dataset

Dataset URL: https://www.kaggle.com/datasets/aakashjoshi123/exercise-and-fitness-metrics-dataset
License(s): Community Data License Agreement - Sharing - Version 1.0
100% 133k/133k [00:00<00:00, 453kB/s]
100% 133k/133k [00:00<00:00, 453kB/s]


In [ ]:
!unzip '/content/exercise-and-fitness-metrics-dataset.zip' -d '/content/dataset/'
!rm '/content/exercise-and-fitness-metrics-dataset.zip'

Archive:  /content/exercise-and-fitness-metrics-dataset.zip
  inflating: /content/dataset/exercise_dataset.csv  


In [ ]:
df = pd.read_csv('/content/dataset/exercise_dataset.csv')
df.head(15)

,ID,Exercise,Calories Burn,Dream Weight,Actual Weight,Age,Gender,Duration,Heart Rate,BMI,Weather Conditions,Exercise Intensity
0,1,Exercise 2,286.959851,91.892531,96.301115,45,Male,37,170,29.426275,Rainy,5
1,2,Exercise 7,343.453036,64.165097,61.104668,25,Male,43,142,21.286346,Rainy,5
2,3,Exercise 4,261.223465,70.846224,71.766724,20,Male,20,148,27.899592,Cloudy,4
3,4,Exercise 5,127.183858,79.477008,82.984456,33,Male,39,170,33.729552,Sunny,10
4,5,Exercise 10,416.318374,89.960226,85.643174,29,Female,34,118,23.286113,Cloudy,3
5,6,Exercise 1,479.722690,78.887578,80.596592,60,Female,41,169,34.719336,Rainy,10
6,7,Exercise 9,457.631361,65.681125,61.815386,18,Male,53,103,34.594641,Cloudy,10
7,8,Exercise 4,272.956982,64.929563,62.806489,42,Male,25,104,22.050103,Cloudy,2
8,9,Exercise 10,195.032273,52.731065,54.537688,49,Male,37,161,30.948847,Sunny,1
9,10,Exercise 8,259.531145,95.164097,97.436833,41,Male,55,103,31.224045,Cloudy,10


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3864 entries, 0 to 3863
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  3864 non-null   int64  
 1   Exercise            3864 non-null   object 
 2   Calories Burn       3864 non-null   float64
 3   Dream Weight        3864 non-null   float64
 4   Actual Weight       3864 non-null   float64
 5   Age                 3864 non-null   int64  
 6   Gender              3864 non-null   object 
 7   Duration            3864 non-null   int64  
 8   Heart Rate          3864 non-null   int64  
 9   BMI                 3864 non-null   float64
 10  Weather Conditions  3864 non-null   object 
 11  Exercise Intensity  3864 non-null   int64  
dtypes: float64(4), int64(5), object(3)
memory usage: 362.4+ KB


In [ ]:
# rounding to 0 decimal places for Heart Rate
df['BMI'] = df['BMI'].round(0)

In [ ]:
# calculate average Heart Rate for each Exercise Intensity
df.groupby('BMI')['Heart Rate'].mean().round(2)

BMI
19.0    142.06
20.0    138.09
21.0    137.63
22.0    138.95
23.0    139.69
24.0    142.57
25.0    137.88
26.0    140.49
27.0    137.32
28.0    138.97
29.0    139.73
30.0    141.08
31.0    139.07
32.0    140.50
33.0    139.30
34.0    140.82
35.0    143.29
Name: Heart Rate, dtype: float64

In [ ]:
df.head(15)

,ID,Exercise,Calories Burn,Dream Weight,Actual Weight,Age,Gender,Duration,Heart Rate,BMI,Weather Conditions,Exercise Intensity
0,1,Exercise 2,286.959851,91.892531,96.301115,45,Male,37,170,29.0,Rainy,5
1,2,Exercise 7,343.453036,64.165097,61.104668,25,Male,43,142,21.0,Rainy,5
2,3,Exercise 4,261.223465,70.846224,71.766724,20,Male,20,148,28.0,Cloudy,4
3,4,Exercise 5,127.183858,79.477008,82.984456,33,Male,39,170,34.0,Sunny,10
4,5,Exercise 10,416.318374,89.960226,85.643174,29,Female,34,118,23.0,Cloudy,3
5,6,Exercise 1,479.722690,78.887578,80.596592,60,Female,41,169,35.0,Rainy,10
6,7,Exercise 9,457.631361,65.681125,61.815386,18,Male,53,103,35.0,Cloudy,10
7,8,Exercise 4,272.956982,64.929563,62.806489,42,Male,25,104,22.0,Cloudy,2
8,9,Exercise 10,195.032273,52.731065,54.537688,49,Male,37,161,31.0,Sunny,1
9,10,Exercise 8,259.531145,95.164097,97.436833,41,Male,55,103,31.0,Cloudy,10


# Fitness Dataset

In [ ]:
# download dataset
!kaggle datasets download -d arashnic/fitbit
!unzip '/content/fitbit.zip' -d '/content/dataset/'
!rm '/content/fitbit.zip'

Dataset URL: https://www.kaggle.com/datasets/arashnic/fitbit
License(s): CC0-1.0
 95% 41.0M/43.3M [00:02<00:00, 27.6MB/s]
100% 43.3M/43.3M [00:02<00:00, 20.5MB/s]
Archive:  /content/fitbit.zip
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/dailyActivity_merged.csv  
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/heartrate_seconds_merged.csv  
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/hourlyCalories_merged.csv  
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/hourlyIntensities_merged.csv  
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/hourlySteps_merged.csv  
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/minuteCaloriesNarrow_merged.csv  
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11

In [ ]:
# Read CSV
df_Intensities = pd.read_csv('/content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/minuteIntensitiesNarrow_merged.csv')
df_HR = pd.read_csv('/content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/heartrate_seconds_merged.csv')
df_Calories = pd.read_csv('/content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/minuteCaloriesNarrow_merged.csv')
df_METs = pd.read_csv('/content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/minuteMETsNarrow_merged.csv')

df_Intensities1 = pd.read_csv('/content/dataset/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/minuteIntensitiesNarrow_merged.csv')
df_HR1 = pd.read_csv('/content/dataset/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/heartrate_seconds_merged.csv')
df_Calories1 = pd.read_csv('/content/dataset/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/minuteCaloriesNarrow_merged.csv')
df_METs1 = pd.read_csv('/content/dataset/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/minuteMETsNarrow_merged.csv')

In [ ]:
# Concatenate df
df_HR = pd.concat([df_HR, df_HR1], ignore_index=True)
df_Intensities = pd.concat([df_Intensities, df_Intensities1], ignore_index=True)
df_Calories = pd.concat([df_Calories, df_Calories1], ignore_index=True)
df_METs = pd.concat([df_METs, df_METs1], ignore_index=True)

In [ ]:
df_Intensities.info()
df_HR.info()
df_Calories.info()
df_METs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2770620 entries, 0 to 2770619
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   Id              int64 
 1   ActivityMinute  object
 2   Intensity       int64 
dtypes: int64(2), object(1)
memory usage: 63.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3638339 entries, 0 to 3638338
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Time    object
 2   Value   int64 
dtypes: int64(2), object(1)
memory usage: 83.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2770620 entries, 0 to 2770619
Data columns (total 3 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Id              int64  
 1   ActivityMinute  object 
 2   Calories        float64
dtypes: float64(1), int64(1), object(1)
memory usage: 63.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2770620 entries, 0 to 2770619
Data columns (total 3 columns):
 #

In [ ]:
# Rename columns
df_HR.rename(columns={'Time': 'ActivityMinute'}, inplace=True)
df_HR.rename(columns={'Value': 'HR'}, inplace=True)

# Merge df
df_all = df_Intensities.merge(df_HR, on=['Id', 'ActivityMinute'], how='inner') \
                       .merge(df_Calories, on=['Id', 'ActivityMinute'], how='inner') \
                       .merge(df_METs, on=['Id', 'ActivityMinute'], how='inner')

df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303199 entries, 0 to 303198
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Id              303199 non-null  int64  
 1   ActivityMinute  303199 non-null  object 
 2   Intensity       303199 non-null  int64  
 3   HR              303199 non-null  int64  
 4   Calories        303199 non-null  float64
 5   METs            303199 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 13.9+ MB


In [ ]:
df_all['Intensity'].value_counts()

Intensity
0    221728
1     67067
3      9140
2      5264
Name: count, dtype: int64

In [ ]:
# Balance the dataset

# feature, target column
X = df_all.drop(columns=['Intensity'])
y = df_all['Intensity']

# undersampling
under_sampling = RandomUnderSampler(random_state=42)
X_us, y_us = under_sampling.fit_resample(X, y)
df_all_resampled = pd.concat([X_us, y_us], axis=1)

# new distribution
print(df_all_resampled['Intensity'].value_counts())
df_all_resampled.info()

Intensity
0    5264
1    5264
2    5264
3    5264
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21056 entries, 0 to 21055
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              21056 non-null  int64  
 1   ActivityMinute  21056 non-null  object 
 2   HR              21056 non-null  int64  
 3   Calories        21056 non-null  float64
 4   METs            21056 non-null  int64  
 5   Intensity       21056 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 987.1+ KB


In [ ]:
df_all_resampled['METs'].mean()

43.19813829787234

In [ ]:
df_all_resampled.groupby('Intensity').agg({
    'HR': 'mean',
    'Calories': 'mean',
    'METs': 'mean'
})

,HR,Calories,METs
Intensity,,,
0,67.973214,1.150020,10.715616
1,86.471315,3.599079,33.902546
2,101.630319,5.648026,51.873290
3,119.462576,8.644752,76.301102


In [ ]:
def q25(x):
    return np.percentile(x, 25)

def q75(x):
    return np.percentile(x, 75)

# Calculate descriptive statistics for HR, Calories, and METs grouped by Intensity
stats = df_all_resampled.groupby('Intensity').agg({
    'HR': ['min', q25, 'mean', q75, 'max'],
    'Calories': ['min', q25, 'mean', q75, 'max'],
    'METs': ['min', q25, 'mean', q75, 'max']
})

# Rename the columns for easier interpretation
stats.columns = ['_'.join(col) for col in stats.columns]

# Display the results
print(stats)


           HR_min  HR_q25     HR_mean  HR_q75  HR_max  Calories_min  \
Intensity                                                             
0              42    60.0   67.973214    74.0     118       0.77510   
1              46    77.0   86.471315    96.0     143       1.96526   
2              58    94.0  101.630319   110.0     163       3.93052   
3              63   109.0  119.462576   128.0     203       5.35980   

           Calories_q25  Calories_mean  Calories_q75  Calories_max  METs_min  \
Intensity                                                                      
0               0.94060       1.150020       1.28997       2.06310        10   
1               2.70218       3.599079       4.14341      15.29077        22   
2               4.97352       5.648026       6.27096       7.97732        43   
3               6.77760       8.644752       9.85686      19.25560        60   

           METs_q25  METs_mean  METs_q75  METs_max  
Intensity                              

In [ ]:
# Separate features and target
X = df_all_resampled[['HR']]#, 'Calories', 'METs']]
y = df_all_resampled['Intensity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
joblib.dump(scaler, 'scaler.pkl')

model = RandomForestClassifier(random_state=42)

# Random Forest Hyperparameter tuning
# param_grid = {                          #ini klo pake HR, Calor, MET
#     'n_estimators': [75, 100, 125],
#     'max_depth': [5, 7, 10],
#     'min_samples_split': [20, 25, 30],
# }
param_grid = {
    'n_estimators': [2,3,5,8,10],
    'max_depth': [2, 3, 5, 7, 8,10],
    'min_samples_split': [2,3,4,5],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters from Grid Search
print("Best parameters found: ", grid_search.best_params_)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Best parameters found:  {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 5}


In [ ]:
# Best parameters
# best_params = {'max_depth': 7, 'min_samples_split': 25, 'n_estimators': 100, 'random_state': 42} #ini klo pake HR, Calor, MET
best_params = {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 10, 'random_state': 42}

# train final model
final_model = RandomForestClassifier(**best_params)
final_model.fit(X_train, y_train)

y_pred = final_model.predict(X_test)

# Eval
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Classification Report: \n", classification_report(y_test, y_pred))

Accuracy:  0.6132478632478633
Classification Report: 
               precision    recall  f1-score   support

           0       0.76      0.76      0.76      1049
           1       0.50      0.47      0.48      1102
           2       0.48      0.52      0.50      1015
           3       0.72      0.71      0.72      1046

    accuracy                           0.61      4212
   macro avg       0.61      0.61      0.61      4212
weighted avg       0.61      0.61      0.61      4212



In [ ]:
# Feature importance
feature_importances = final_model.feature_importances_
features = ['HR']#, 'METs', 'HR']
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

# Save model
import joblib
joblib.dump(final_model, 'final_model.pkl')

# from joblib import dump, load

# Save the model to a file
joblib.dump(final_model, 'final_model.joblib')


  Feature  Importance
0      HR         1.0


['final_model.joblib']

In [ ]:
# Load the model from the file
loaded_model = joblib.load('final_model.joblib')


In [ ]:
 pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 25.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
     

In [ ]:
tensorflowjsconverter \
    --input_format=final_model \
    --output_node_names='MobilenetV1/Predictions/Reshape_1' \
    --saved_model_tags=serve \
    /mobilenet/saved_model \
    /mobilenet/web_model

SyntaxError: cannot assign to expression (<ipython-input-40-347e9d5bb450>, line 1)

In [ ]:
import json
from sklearn import datasets

In [ ]:
# save final_model to json
model_params = final_model.get_params()

with open('model_params.json', 'w') as json_file:
    json.dump(model_params, json_file)

In [ ]:
import json
from sklearn.ensemble import RandomForestClassifier

# Load model parameters from JSON
with open('model_params.json', 'r') as json_file:
    model_params = json.load(json_file)

# Recreate the model with loaded parameters
loaded_model = RandomForestClassifier(**model_params)


# For Demo, we only use HR as variable, because we dont have smartwatch to get the real data.
Hr = 110
# Calories = Hr / np.random.uniform(14, 60)
# METs = Hr / np.random.uniform(1.5, 8)

print(f"HR: {Hr}")#, Calories: {Calories}, METs: {METs}")

# Example new input data
new_data = pd.DataFrame({
    'HR': [Hr]#,
    # 'Calories': [Calories],
    # 'METs': [METs]
})

# Scale the new input data
new_data_scaled = scaler.transform(new_data)

# Train or use the loaded model as needed
loaded_model.predict(new_data_scaled)  # Example usage


In [ ]:
final_model = joblib.load('final_model.pkl')
scaler = joblib.load('scaler.pkl')

# For Demo, we only use HR as variable, because we dont have smartwatch to get the real data.
Hr = 110
# Calories = Hr / np.random.uniform(14, 60)
# METs = Hr / np.random.uniform(1.5, 8)

print(f"HR: {Hr}")#, Calories: {Calories}, METs: {METs}")

# Example new input data
new_data = pd.DataFrame({
    'HR': [Hr]#,
    # 'Calories': [Calories],
    # 'METs': [METs]
})

# Scale the new input data
new_data_scaled = scaler.transform(new_data)

# Make predictions
predicted_intensity = final_model.predict(new_data_scaled)

print(f"Predicted Intensity: {predicted_intensity[0]}")


In [ ]:
# Load the trained model and scaler
final_model = joblib.load('final_model.pkl')
scaler = joblib.load('scaler.pkl')

def recommend_songs(heart_rate, df_assign):
    """Recommends songs based on heart rate and preprocessed DataFrame.

    Args:
        heart_rate (int): The user's current heart rate.
        df_assign (pd.DataFrame): DataFrame with 'track_id', 'track_name',
                                    'popularity', and 'intensity' columns.

    Returns:
        pd.DataFrame: DataFrame containing recommended songs.
    """

    # Predict intensity from heart rate
    new_data = pd.DataFrame({'HR': [heart_rate]})
    new_data_scaled = scaler.transform(new_data)
    predicted_intensity = final_model.predict(new_data_scaled)[0]

    print(f"Predicted Intensity: {predicted_intensity}")

    # Filter songs based on predicted intensity
    recommended_df = df_assign[df_assign['intensity'].apply(lambda x: predicted_intensity in x)].copy()

    # Apply popularity weighting
    recommended_df['selection_probability'] = recommended_df['popularity'].apply(
        lambda pop: 0.8 if pop > 60 else (0.2 if pop > 40 else 0.1)
    )

    # Sample songs based on weighted probability
    return recommended_df.sample(n=min(1, len(recommended_df)),  # Recommend up to 5 songs
                                  weights='selection_probability')

# Example usage:
hr = 120
recommendations = recommend_songs(hr, df_assign)
print(recommendations[['track_name', 'track_genre', 'popularity', 'intensity']])

In [ ]:
# Load the model and scaler
final_model = joblib.load('final_model.pkl')
scaler = joblib.load('scaler.pkl')

# Example heart rate
Hr = 120
print(f"HR: {Hr}")

# Prepare the new data for prediction
new_data = pd.DataFrame({'HR': [Hr]})
new_data_scaled = scaler.transform(new_data)

# Make predictions
predicted_intensity = final_model.predict(new_data_scaled)
predicted_intensity = int(predicted_intensity[0])
print(f"Predicted Intensity: {predicted_intensity}")

# Define the selection criteria based on popularity
def choose_song_based_on_popularity(data, intensity, popularity_thresholds):
    filtered_songs = data[data['intensity'].apply(lambda x: intensity in x)].copy()

    # Define the probability weights based on popularity
    filtered_songs['probability'] = filtered_songs['popularity'].apply(
        lambda x: 0.7 if x > popularity_thresholds['high'] else (0.2 if popularity_thresholds['low'] < x <= popularity_thresholds['high'] else 0.1)
    )

    # Normalize probabilities
    filtered_songs['probability'] /= filtered_songs['probability'].sum()

    # Choose a song based on the defined probabilities
    chosen_song = filtered_songs.sample(weights=filtered_songs['probability']).iloc[0]

    return chosen_song

# Define popularity thresholds
popularity_thresholds = {
    'high': 60,
    'low': 55
}

# Choose a song based on the predicted intensity and popularity
chosen_song = choose_song_based_on_popularity(df_assign, predicted_intensity, popularity_thresholds)

print(f"Chosen Song: {chosen_song['track_name']} by ID: {chosen_song['track_id']}; {chosen_song['popularity']}, {chosen_song['intensity']}")


In [ ]:
def recommend_songs(predicted_intensity, df_assign, num_songs=50):
    """Recommends songs based on predicted intensity and popularity.

    Args:
        predicted_intensity (int): The predicted intensity level (0-3).
        df_assign (pd.DataFrame): DataFrame with 'track_id', 'track_name',
                                    'popularity', and 'intensity' columns.
        num_songs (int, optional): Number of songs to recommend. Defaults to 50.

    Returns:
        pd.DataFrame: DataFrame with recommended songs.
    """

    # Filter songs matching predicted intensity
    recommended_df = df_assign[df_assign['intensity'].apply(lambda x: predicted_intensity in x)].copy()

    # Sort by popularity (descending)
    recommended_df = recommended_df.sort_values(by='popularity', ascending=False)

    # Apply popularity-based weighting (adjust percentages as needed)
    high_pop = recommended_df[recommended_df['popularity'] > 60].sample(frac=0.3)
    mid_pop = recommended_df[(recommended_df['popularity'] <= 60) &
                              (recommended_df['popularity'] > 55)].sample(frac=0.35)
    low_pop = recommended_df[recommended_df['popularity'] <= 55].sample(frac=0.3)

    # Combine weighted samples
    recommended_df = pd.concat([high_pop, mid_pop, low_pop])

    # Return top 'num_songs' recommendations
    return recommended_df.head(num_songs)

# --- MAIN EXECUTION BLOCK ---

if __name__ == "__main__":
    final_model = joblib.load('final_model.pkl')
    scaler = joblib.load('scaler.pkl')

    # ... (Your code to get user's heart rate and predict intensity)

    # Example usage:
    predicted_intensity = 2  # Replace with your actual prediction
    recommendations = recommend_songs(predicted_intensity, df_assign, num_songs=1)

    print("Recommended Songs:")
    print(recommendations[['track_name', 'track_genre', 'popularity']])

In [ ]:
# Function to recommend songs
def recommend_songs(predicted_intensity, num_recommendations=50):
    recommended_df = df_assign[df_assign['intensity'].apply(lambda x: predicted_intensity in x)].copy()

    # If no songs match the exact intensity, expand to adjacent intensities
    if recommended_df.empty:
        if predicted_intensity == 0:
            recommended_df = df_assign[df_assign['intensity'].apply(lambda x: 1 in x)].copy()
        elif predicted_intensity == 3:
            recommended_df = df_assign[df_assign['intensity'].apply(lambda x: 2 in x)].copy()
        else:
            recommended_df = df_assign[df_assign['intensity'].apply(lambda x: predicted_intensity - 1 in x or predicted_intensity + 1 in x)].copy()

    # Apply popularity-based weighting for random selection
    recommended_df['weight'] = recommended_df['popularity'].apply(
        lambda pop: 0.7 if pop > 60 else (0.2 if 55 < pop <= 60 else 0.1)
    )

    # If not enough songs after filtering and weighting, recommend whatever is available
    if len(recommended_df) < num_recommendations:
        num_recommendations = len(recommended_df)

    # Randomly select songs based on weighted probability
    recommended_songs = recommended_df.sample(n=num_recommendations, weights='weight')

    return recommended_songs[['track_name', 'track_genre', 'popularity']]

Hr = 120

# Create DataFrame for prediction
new_data = pd.DataFrame({
    'HR': [Hr],
    # 'Calories': [Calories],
    # 'METs': [METs]
})

# Scale input and predict intensity
new_data_scaled = scaler.transform(new_data)
predicted_intensity = final_model.predict(new_data_scaled)[0]

# Get song recommendations
recommendations = recommend_songs(predicted_intensity,5)
print(recommendations)